In [ ]:
import requests
import pandas as pd
import time
import json
import shelve

# Your Google Places API key
API_KEY = ''

def fetch_places(lat, lng, radius, page_token=None):
    with shelve.open('places_cache.db') as cache:
        cache_key = f'{lat},{lng},{radius},{page_token}'
        
        
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&key={}'.format(lat, lng, radius, API_KEY)
        if page_token:
            url += '&pagetoken={}'.format(page_token)
        print(url)     
                
                
        if cache_key in cache:
            res_json = cache[cache_key]
        else:
            url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&key={}'.format(lat, lng, radius, API_KEY)
            if page_token:
                url += '&pagetoken={}'.format(page_token)
            response = requests.get(url)
            res_json = response.json()
            # Dump the API call and its output
            with open('api_calls.json', 'a') as f:
                json.dump({
                    'url': url,
                    'response': res_json
                }, f)
                f.write('\n')  # Add newline for next entry
            cache[cache_key] = res_json
    if 'results' in res_json and res_json['results']:
        return res_json
    else:
        print("else statement")
        return None


def get_places(lat, lng, radius):
    page_token = None
    res_json = fetch_places(lat, lng, radius, page_token)
    while res_json is not None:
        print(f"Lat: {lat}, Lng: {lng}, Radius: {radius}, Page Token: {page_token}")
        for place in res_json['results']:
            yield place
        page_token = res_json.get('next_page_token')
        if page_token:
            time.sleep(2.5)  # Sleep for 3 seconds to respect rate limit
            res_json = fetch_places(lat, lng, radius, page_token)
        else:
            print("none")
            res_json = None




def get_coordinates():
    return [(40.7189838, -73.9893590)]
# The latitude and longitude coordinates along Delancey Street
# These coordinates would need to be calculated to span the entire length of Delancey Street between Mulberry Street and Pitt Street
# coordinates = [
#     (40.7189838, -73.9893590),  # Example coordinates, replace with real ones
#     # ...
# ]

coordinates = get_coordinates()

radius = .022 * 1609.34  # 0.2 miles in meters

places = []

# Loop through all the coordinates and get places
for lat, lng in coordinates:
    places.extend(list(get_places(lat, lng, radius)))

# Convert the list of places to a DataFrame
df = pd.DataFrame(places)

# Remove duplicates
df = df.drop_duplicates(subset=['name', 'vicinity'])
# Remove permanently closed
df = df[df['business_status'] != 'PERMANENTLY_CLOSED']


# Save the DataFrame to a CSV file
df.to_csv('places.csv', index=False)